In [1]:
%cd ..

/home/nikita/edu/competitions/admet


In [2]:
import pandas as pd
import datamol as dm
import numpy as np
import warnings

from molfeat.calc import RDKitDescriptors2D, FPCalculator, MordredDescriptors
from molfeat.trans import MoleculeTransformer

from molfeat.trans.concat import FeatConcat
from molfeat.trans.fp import FPVecTransformer

from rdkit import Chem, RDLogger
from rdkit.Chem import Descriptors
from rdkit.Chem.rdFingerprintGenerator import GetMorganGenerator


from mordred import Calculator, descriptors
import rdkit


from tqdm import tqdm

warnings.filterwarnings("ignore")

In [4]:
train_data = dm.read_csv("data/shuffled_final_extended_train_data.csv", index_col=0, smiles_column="Drug")

[08:47:22] SMILES Parse Error: extra open parentheses for input: 'COP(=S)(OCC)Oc1ccc(N)c(CCС)c1'
[08:47:22] SMILES Parse Error: extra open parentheses for input: 'COC(=O)C1=C(CCС)NC(CCС)=C(C(=O)OC)C1c1ccccc1[N+](=O)[O-]'
[08:47:22] SMILES Parse Error: extra open parentheses for input: 'Cc1ccc(CCС)o1'
[08:47:22] SMILES Parse Error: extra open parentheses for input: 'C=CCOc1ccccc1OCC(O)CNC(CCС)C'
[08:47:22] SMILES Parse Error: extra open parentheses for input: 'CC1=C(NC2=CC=CC=C12)CN(CCС)C3=NC(=NC(=C3)C4CC(C4)O)N'
[08:47:22] SMILES Parse Error: extra open parentheses for input: 'CN(CCС)CCCC1c2ccccc2Nc2ccc(Cl)cc21'
[08:47:22] SMILES Parse Error: extra open parentheses for input: 'CC1COc2c(N3CCN(CCС)CC3)c(F)c(C(=O)O)c3c(=O)ccn1c23'
[08:47:22] SMILES Parse Error: extra open parentheses for input: 'CN(CCС)c1nc(N(CCС)C)nc(N(CCС)C)n1'
[08:47:22] SMILES Parse Error: extra open parentheses for input: 'CCN(CCС)CCNC(=O)c1cc(Cl)cc(Cl)c1OC'
[08:47:22] SMILES Parse Error: extra open parentheses for i

In [4]:
d2 = FPVecTransformer("desc2D", n_jobs=8, dtype=np.float32, replace_nan=True)
maccs = FPVecTransformer("maccs", dtype=np.float32)
ecfp4 = FPVecTransformer("ecfp:4", dtype=np.float32)

# Wrap the calculator in a transformer instance
featurizer = FeatConcat([maccs, ecfp4, d2], dtype=np.float32)

with dm.without_rdkit_log():
    feats = featurizer(train_data["mol"])

/home/nikita/edu/competitions/admet/.conda/lib/python3.11/site-packages/molfeat/calc/descriptors.py:46: RuntimeWarning: All-NaN slice encountered
  min_charge, max_charge = np.nanmin(atomic_charges), np.nanmax(atomic_charges)
[07:55:07] DEPRECATION WARNING: please use MorganGenerator
[07:55:07] DEPRECATION WARNING: please use MorganGenerator
[07:55:07] DEPRECATION WARNING: please use MorganGenerator
[07:55:07] DEPRECATION WARNING: please use MorganGenerator
[07:55:07] DEPRECATION WARNING: please use MorganGenerator
[07:55:07] DEPRECATION WARNING: please use MorganGenerator
[07:55:07] DEPRECATION WARNING: please use MorganGenerator
[07:55:07] DEPRECATION WARNING: please use MorganGenerator
[07:55:07] DEPRECATION WARNING: please use MorganGenerator
[07:55:07] DEPRECATION WARNING: please use MorganGenerator
[07:55:07] DEPRECATION WARNING: please use MorganGenerator
[07:55:07] DEPRECATION WARNING: please use MorganGenerator
[07:55:07] DEPRECATION WARNING: please use MorganGenerator
[07:55:

In [5]:
warnings.filterwarnings("ignore")
RDLogger.DisableLog("rdApp.*")


def get_decsriptors_df(smiles_list):
    descriptors_list = []

    for smiles in tqdm(smiles_list):
        descriptors_list.append(
            Descriptors.CalcMolDescriptors(Chem.MolFromSmiles(smiles), 0)
        )
    return pd.DataFrame(descriptors_list).fillna(0)

rdkit_feats = get_decsriptors_df(train_data["Drug"])

100%|██████████| 1221/1221 [00:10<00:00, 116.42it/s]


In [6]:
calc = Calculator(descriptors, ignore_3D=False)
mols = [rdkit.Chem.MolFromSmiles(smiles) for smiles in train_data["Drug"]]
desc_vector = calc.pandas(mols).astype(float).fillna(0)

100%|██████████| 1221/1221 [02:14<00:00,  9.06it/s]


In [7]:
def fp_str_to_array(str_): 
    return np.array([int(bit) for bit in list(str_)]) 
 
def get_decsriptors_w_fp_df(smiles_list): 
    descriptors_list = [] 
    fpgen = GetMorganGenerator(radius=4, 
                               fpSize=64) 
 
    for smiles in tqdm(smiles_list): 
        fp = fp_str_to_array(fpgen.GetFingerprint(Chem.MolFromSmiles(smiles)).ToBitString()) 
        descriptors_list.append(list(Descriptors.CalcMolDescriptors( 
            Chem.MolFromSmiles(smiles), 0).values()) + fp.tolist()) 
    return pd.DataFrame(descriptors_list).fillna(0)

In [8]:
fingerprints = get_decsriptors_w_fp_df(train_data["Drug"].tolist())

100%|██████████| 1221/1221 [00:13<00:00, 92.86it/s]


In [9]:
feats = pd.DataFrame(feats, columns=[f"{i}" for i in range(feats.shape[1])])

In [10]:
total_data = pd.concat([train_data, feats, rdkit_feats, desc_vector, fingerprints], axis=1)
total_data.drop(["mol"], axis=1, inplace=True)

In [11]:
total_data

,Drug_ID,Drug,property,0,1,2,3,4,5,6,...,264,265,266,267,268,269,270,271,272,273
0,0,Cc1cc(=O)oc2ccc3c(c12)C1(C)OOC1(C)O3,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,1,0,1,0,1,0
1,1,CN(CC(O)CO)N=O,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,0,0,1,0,0,0,0,0,0
2,2,O=[N+]([O-])c1ccc(OC2CO2)cc1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,1,0,0,1,1,0,1,0,0
3,3,Cn1cnc2cc([N+](=O)[O-])ccc21,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,1,1,0,0,0,0,0
4,4,CC[C@H](CO)NCCN[C@H](CC)CO,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1216,1216,CCN(CC)C(=O)COc1cc2c(O)c3c(O)c(C)c4c(c13)C(=O)...,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,1,0,1,1,1,1,1,1,1
1217,1217,O=C(Cc1ccc(Cl)c(Cl)c1)N1CCc2sccc2[C@H]1CN1CCCC1,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,1,0,0,1,1,1,1,0,1
1218,1218,CN1CC[C@]23c4c5ccc(O)c4O[C@H]2C(=O)CC[C@H]3[C@...,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,1,1,1,1,1,1,1,1,0
1219,1219,Cn1nc(-c2cccc(F)c2)n(C)c1=S,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,1,1,1,1,0,1,0


In [12]:
total_data.to_csv("data/test_data_with_all_new_features.csv", index=False)